In [ ]:
pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/cleaned_total_reviews.csv')
df.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative,"['not', 'smoothest', 'easiest', 'apps', 'navig..."
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral,"['app', 'has', 'good', 'features', 'eg', 'let'..."
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive,"['easy', 'app', 'use', '.', 'easy', 'navigate'..."
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative,"['pointless', 'app', '.', 'i', 'can', 'see', '..."
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive,"['update', '.', 'issue', 'was', 'resolved', 'q..."


In [ ]:
# Label mapping
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df['encoded_labels'] = df['manual_labels'].map(label_map)

In [ ]:
# Separating classes for resampling
df_neg = df[df.encoded_labels == label_map['Negative']]
df_neu = df[df.encoded_labels == label_map['Neutral']]
df_pos = df[df.encoded_labels == label_map['Positive']]

In [ ]:
# Resampling
df_pos_upsampled = resample(df_pos, replace=True, n_samples=len(df_neg), random_state=123)
df_neu_upsampled = resample(df_neu, replace=True, n_samples=len(df_neg), random_state=123)

In [ ]:
# Combine into a single DataFrame
df_resampled = pd.concat([df_neg, df_pos_upsampled, df_neu_upsampled])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def encode_reviews(review):
    encoded_batch = tokenizer(review, padding=True, truncation=True, max_length=128, return_tensors="tf")
    return encoded_batch['input_ids'].numpy(), encoded_batch['attention_mask'].numpy()

In [ ]:
input_ids, attention_masks = encode_reviews(df_resampled['review'].tolist())

In [ ]:
# Train-test split
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids, df_resampled['encoded_labels'].values, test_size=0.2, random_state=0
)

In [ ]:
train_masks, test_masks, _, _ = train_test_split(
    attention_masks, df_resampled['encoded_labels'].values, test_size=0.2, random_state=0
)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(({'input_ids': train_inputs, 'attention_mask': train_masks}, train_labels)).shuffle(100).batch(32)
test_data = tf.data.Dataset.from_tensor_slices(({'input_ids': test_inputs, 'attention_mask': test_masks}, test_labels)).batch(32)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, epochs= 3, validation_data=test_data)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
86/86 [==============================] - 4300s 49s/step - loss: 6.2717 - accuracy: 0.3394 - val_loss: 6.2616 - val_accuracy: 0.3120
Epoch 2/3
86/86 [==============================] - 4150s 48s/step - loss: 6.0169 - accuracy: 0.3387 - val_loss: 6.2616 - val_accuracy: 0.3120
Epoch 3/3
86/86 [==============================] - 4154s 48s/step - loss: 6.0169 - accuracy: 0.3387 - val_loss: 6.2616 - val_accuracy: 0.3120


In [ ]:
eval_loss, eval_accuracy = model.evaluate(test_data)
print(f"Test Loss: {eval_loss}, Test Accuracy: {eval_accuracy}")

22/22 [==============================] - 303s 14s/step - loss: 6.2616 - accuracy: 0.3120
Test Loss: 6.261643409729004, Test Accuracy: 0.3119533658027649


In [ ]:
# Predict the test dataset
predictions = model.predict(test_data)
# Convert the prediction logits to class indices
predicted_classes = np.argmax(predictions.logits, axis=1)

22/22 [==============================] - 349s 15s/step


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Extract labels from the test dataset
actual_labels = np.concatenate([y for x, y in test_data], axis=0)

In [ ]:
print(classification_report(actual_labels, predicted_classes, target_names=['Negative', 'Neutral', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       235
     Neutral       0.00      0.00      0.00       237
    Positive       0.31      1.00      0.48       214

    accuracy                           0.31       686
   macro avg       0.10      0.33      0.16       686
weighted avg       0.10      0.31      0.15       686



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
